adapting codefrom : https://www.kaggle.com/hitesh1724/titanic-1-fastai-beginner-tutorial

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
import fastcore, fastai
fastcore.__version__, fastai.__version__

('1.3.10', '2.1.8')

In [3]:
df_test = pd.read_csv('../input/titanic/test.csv')
df_train = pd.read_csv('../input/titanic/train.csv')

df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [5]:
# looking at null values in our data
df_train.isnull().sum().sort_index()/len(df_train)

Age            0.198653
Cabin          0.771044
Embarked       0.002245
Fare           0.000000
Name           0.000000
Parch          0.000000
PassengerId    0.000000
Pclass         0.000000
Sex            0.000000
SibSp          0.000000
Survived       0.000000
Ticket         0.000000
dtype: float64

In [6]:
df_train.dtypes
g_train =df_train.columns.to_series().groupby(df_train.dtypes).groups
g_train

{int64: ['PassengerId', 'Survived', 'Pclass', 'SibSp', 'Parch'], float64: ['Age', 'Fare'], object: ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']}

In [7]:
cat_names= [
        'Name', 'Sex', 'Ticket', 'Cabin', 
        'Embarked'
]

cont_names = [ 
    'PassengerId', 'Pclass', 'SibSp', 'Parch', 
    'Age', 'Fare'
 ]

In [8]:
from fastai.tabular.all import *

In [9]:
splits = RandomSplitter(valid_pct=0.2)(range_of(df_train))

to = TabularPandas(df_train, procs=[Categorify, FillMissing, Normalize],
                   cat_names = cat_names,
                   cont_names = cont_names,
                   y_names = 'Survived',
                   splits = splits
                   )

In [10]:
g_train = to.train.xs.columns.to_series().groupby(to.train.xs.dtypes).groups

In [11]:
g_train

{int8: ['Sex', 'Embarked', 'Age_na'], int16: ['Name', 'Ticket', 'Cabin'], float64: ['PassengerId', 'Pclass', 'SibSp', 'Parch', 'Age', 'Fare']}

In [12]:
to.train

     PassengerId  Survived    Pclass  Name  Sex       Age     SibSp     Parch  \
85     -1.393272         1  0.837609    53    1  0.268604  2.248820 -0.472399   
401    -0.171212         0  0.837609     7    2 -0.272924 -0.473436 -0.472399   
364    -0.314302         0  0.837609   600    2 -0.118202  0.433983 -0.472399   
117    -1.269519         0 -0.359215   827    2 -0.040841  0.433983 -0.472399   
645     0.772403         1 -1.556039   333    2  1.429021  0.433983 -0.472399   
..           ...       ...       ...   ...  ...       ...       ...       ...   
121    -1.254050         0  0.837609   552    2 -0.118202 -0.473436 -0.472399   
32     -1.598237         1  0.837609   290    1 -0.118202 -0.473436 -0.472399   
201    -0.944668         0  0.837609   722    2 -0.118202  6.785912  1.924900   
492     0.180710         0 -1.556039   548    2  1.970550 -0.473436 -0.472399   
521     0.292861         0  0.837609   842    2 -0.582369 -0.473436 -0.472399   

     Ticket      Fare  Cabi

In [13]:
to.train.xs

,Name,Sex,Ticket,Cabin,Embarked,Age_na,PassengerId,Pclass,SibSp,Parch,Age,Fare
85,53,1,248,0,3,1,-1.393272,0.837609,2.248820,-0.472399,0.268604,-0.340344
401,7,2,293,0,3,1,-0.171212,0.837609,-0.473436,-0.472399,-0.272924,-0.502501
364,600,2,460,0,2,2,-0.314302,0.837609,0.433983,-0.472399,-0.118202,-0.347621
117,827,2,54,0,3,1,-1.269519,-0.359215,0.433983,-0.472399,-0.040841,-0.233280
645,333,2,588,102,1,1,0.772403,-1.556039,0.433983,-0.472399,1.429021,0.925291
...,...,...,...,...,...,...,...,...,...,...,...,...
121,552,2,541,0,3,2,-1.254050,0.837609,-0.473436,-0.472399,-0.118202,-0.502501
32,290,1,289,0,2,2,-1.598237,0.837609,-0.473436,-0.472399,-0.118202,-0.508738
201,722,2,569,0,3,2,-0.944668,0.837609,6.785912,1.924900,-0.118202,0.776040
492,548,2,42,65,3,1,0.180710,-1.556039,-0.473436,-0.472399,1.970550,-0.035781


In [14]:
# looking at target values
to.train.ys.values.ravel()

array([1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,

In [15]:
from sklearn.ensemble import RandomForestClassifier

X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_valid, y_valid = to.valid.xs, to.valid.ys.values.ravel()

In [16]:
rnf_classifier = RandomForestClassifier(n_estimators=100, n_jobs=-1)
rnf_classifier.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1)

In [17]:
y_pred = rnf_classifier.predict(X_valid)

from sklearn.metrics import accuracy_score

accuracy_score(y_pred, y_valid)

0.8202247191011236

### getting to test data

In [18]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [19]:
test = TabularPandas(df_test, procs=[Categorify, FillMissing, Normalize],
                     cat_names=cat_names,
                     cont_names=cont_names)

In [20]:
X_test = test.train.xs

In [21]:
X_test.head()

,Name,Sex,Ticket,Cabin,Embarked,Age_na,Fare_na,PassengerId,Pclass,SibSp,Parch,Age,Fare
0,207,2,153,0,2,1,1,-1.727912,0.873482,-0.499470,-0.400248,0.386231,-0.497413
1,404,1,222,0,3,1,1,-1.719625,0.873482,0.616992,-0.400248,1.371369,-0.512278
2,270,2,74,0,2,1,1,-1.711337,-0.315819,-0.499470,-0.400248,2.553536,-0.464100
3,409,2,148,0,3,1,1,-1.703050,0.873482,-0.499470,-0.400248,-0.204852,-0.482475
4,179,1,139,0,3,1,1,-1.694763,0.873482,0.616992,0.619896,-0.598908,-0.417491


In [22]:
X_test.dtypes, len(X_test.dtypes)

(Name             int16
 Sex               int8
 Ticket           int16
 Cabin             int8
 Embarked          int8
 Age_na            int8
 Fare_na           int8
 PassengerId    float64
 Pclass         float64
 SibSp          float64
 Parch          float64
 Age            float64
 Fare           float64
 dtype: object,
 13)

we drop Fare_na , because then our model will match with original 12 features in dataset

In [23]:
X_test = X_test.drop('Fare_na', axis=1)

In [24]:
y_pred = rnf_classifier.predict(X_test)

In [25]:
y_pred = y_pred.astype(int)

In [26]:
output = pd.DataFrame({'PassengerId': df_test.PassengerId, 'Survived': y_pred})
output.to_csv('my_submission_titanic.csv', index=False)
output.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
